# Create Chat Invitations

To run, make sure you have your client PW stored as an env var locally, or replace with a hardcoded string. Also make sure to modify the `venue_ids` var with the list of venues you want to enable chat in.

In [18]:
# Imports

from openreview.api import Invitation
from openreview.api import OpenReviewClient
from openreview import tools
from os import getenv

In [19]:
# Initialize clients

local_client = OpenReviewClient(baseurl='http://localhost:3001', username='OpenReview.net', password=getenv('OR_SUPER_USER_PW'))
# dev_client = OpenReviewClient(baseurl='https://devapi2.openreview.net', username='OpenReview.net', password=getenv('OR_DEV_SUPER_USER_PW'))
# live_client = OpenReviewClient(baseurl='https://api2.openreview.net', username='OpenReview.net', password=getenv('OR_SUPER_USER_PW'))

In [20]:
# Config vars

venue_ids = ['ICML.cc/2023/Conference']
client = local_client

In [22]:
# Post invitation edits

for venue_id in venue_ids:
    # Post forum tab config
    print('Posting submission invitation edit')
    client.post_invitation_edit(
        invitations=f'{venue_id}/-/Edit',
        readers = [venue_id],
        writers = [venue_id],
        signatures = [venue_id],
        invitation = Invitation(
            id = f'{venue_id}/-/Submission',
            reply_forum_views = [
                {
                    'id': 'all',
                    'label': 'All'
                },
                {
                    'id': 'discussion',
                    'label': 'Discussion',
                    'filter': f'-invitations:{venue_id}/Submission${{note.number}}/-/Chat',
                    'nesting': 3,
                    'sort': 'date-desc',
                    'layout': 'default',
                },
                {
                    'id': 'reviewers-chat',
                    'label': 'Reviewers Chat',
                    'filter': f'invitations:{venue_id}${{note.number}}/-/Chat,{venue_id}${{note.number}}/-/Official_Review',
                    'nesting': 1,
                    'sort': 'date-asc',
                    'layout': 'chat',
                    'live': True,
                    'expandedInvitations': [f'{venue_id}/Submission${{note.number}}/-/Chat']
                }
            ]
        )
    )

    # Get all submissions
    submission_notes = client.get_notes(invitation=f'{venue_id}/-/Submission')

    def post(note):
        submission_group_id = f'{venue_id}/Submission{note.number}'
        return client.post_invitation_edit(
            invitations = f'{venue_id}/-/Edit',
            readers = [venue_id],
            writers = [venue_id],
            signatures = [venue_id],
            invitation = Invitation(
                id = f'{submission_group_id}/-/Chat',
                readers = ['everyone'],
                writers = [venue_id],
                signatures = [venue_id],
                invitees = [
                    f'{venue_id}/Program_Chairs', 
                    f'{submission_group_id}/Area_Chairs', 
                    f'{submission_group_id}/Reviewers'
                ],
                edit = {
                    'readers': [venue_id, '${2/signatures}'],
                    'writers': [venue_id],
                    'signatures': {
                        'param': {
                            'enum': [
                                f'{venue_id}/Program_Chairs',
                                f'{submission_group_id}/Area_Chair_.*',
                                f'{submission_group_id}/Reviewer_.*',
                            ]
                        }
                    },
                    'note': {
                        'id': {
                            'param': {
                                'withInvitation': f'{submission_group_id}/-/Chat',
                                'optional': True
                            }
                        },
                        'readers': [
                            f'{venue_id}/Program_Chairs', 
                            f'{submission_group_id}/Area_Chairs', 
                            f'{submission_group_id}/Reviewers'
                        ],
                        'writers': [venue_id],
                        'signatures': ['${3/signatures}'],
                        'ddate': {
                            'param': {
                                'range': [ 0, 9999999999999 ],
                                'optional': True,
                                'deletable': True
                            }
                        },
                        'forum': note.id,
                        'replyto': {
                            'param': {
                                'withForum': note.id
                            }
                        },
                        'content': {
                            'message': {
                                'value': {
                                    'param': {
                                        'type': 'string',
                                        'maxLength': 5000,
                                        'markdown': True
                                    }
                                }
                            }
                        }
                    }
                }
            )
        )             
    
    tools.concurrent_requests(post, submission_notes, 'Posting chat invitations for submissions')
   


Posting submission invitation edit...


Posting chat invitations for submissions...: 100%|██████████| 1000/1000 [00:10<00:00, 98.26it/s]
